# how to determine which parameters mostly affect a result? *A heart disease study*

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import pandas_profiling as profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

C:\Users\ophir\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ophir\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\ophir\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# dataset loading
df = pd.read_csv('../proposal/heart.csv')
# get a summary of the data
print(df.describe().T)

           count        mean        std    min    25%    50%    75%    max
age       1025.0   54.434146   9.072290   29.0   48.0   56.0   61.0   77.0
sex       1025.0    0.695610   0.460373    0.0    0.0    1.0    1.0    1.0
cp        1025.0    0.942439   1.029641    0.0    0.0    1.0    2.0    3.0
trestbps  1025.0  131.611707  17.516718   94.0  120.0  130.0  140.0  200.0
chol      1025.0  246.000000  51.592510  126.0  211.0  240.0  275.0  564.0
fbs       1025.0    0.149268   0.356527    0.0    0.0    0.0    0.0    1.0
restecg   1025.0    0.529756   0.527878    0.0    0.0    1.0    1.0    2.0
thalach   1025.0  149.114146  23.005724   71.0  132.0  152.0  166.0  202.0
exang     1025.0    0.336585   0.472772    0.0    0.0    0.0    1.0    1.0
oldpeak   1025.0    1.071512   1.175053    0.0    0.0    0.8    1.8    6.2
slope     1025.0    1.385366   0.617755    0.0    1.0    1.0    2.0    2.0
ca        1025.0    0.754146   1.030798    0.0    0.0    0.0    1.0    4.0
thal      1025.0    2.323

### Exploratory data analysis

In [3]:
# finding out how many people are sick and how many people are healthy
print(df.target.value_counts())

# plotting a bar chart.
fig = df.target.value_counts().plot(kind = 'bar', color=["red", 'blue'])
fig.set_xticklabels(labels=['Sick', "Healthy"], rotation=0)
plt.title("Disease distibution along the dataset")
plt.ylabel("Amount of people")

1    526
0    499
Name: target, dtype: int64


Text(0, 0.5, 'Amount of people')

##### We can see that the dataset is quite balanced with:
- 526 sick personal 
- 499 healthy personal

In [4]:
# finding out the distribution of gender
labels = 'Male', 'Female'
explode = (0, 0)

fig1, ax1 = plt.subplots()
ax1.pie(df.sex.value_counts(), explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.show()

C:\Users\ophir\AppData\Local\Temp/ipykernel_17744/3221799808.py:9: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


##### We can see that the dataset consists of:
- 69.6% male  
- 30.4% female

In [5]:
# figuring out chest pain type distribution
fig = df.cp.value_counts().plot(kind = 'bar', color = ['green', 'cyan', 'yellow', 'red'])
fig.set_xticklabels(labels=['type 0', 'type 1', 'type 2', 'type 3'], rotation=0)
plt.title('Chest pain type count')

Text(0.5, 1.0, 'Chest pain type count')

#### Data profile:

In [6]:
profiling.ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Data pre-processing

In [ ]:
### Null values
df.isna().sum()

##### There are no null values 

### Machine learning

In [ ]:
# separating features and labels
features = df.iloc[:, 0:-1]
labels = df.iloc[:, -1]

In [ ]:
# spltting the dataset into train and test set
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state=1)

In [ ]:
# logistic regression classifier
lr = LogisticRegression(max_iter = 1000, random_state=1)
lr.fit(x_train, y_train)
lr_score = lr.score(x_test, y_test)
print("Score: " + str(lr_score))
print("Coefficients: " + str(lr.coef_))
